In [1]:
import os
import wandb
import torch
import torch.nn as nn
import random
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
import torch.optim as optim
import torch.nn.functional as Function
import argparse


In [ ]:
SYMBOL_BEGIN, SYMBOL_END, SYMBOL_UNKNOWN, SYMBOL_PADDING = 0, 1, 2, 3

INPUT_LABEL = "input"
TARGET_LABEL = "target"
DELIMETER = ","

RNN_KEY = "RNN"
GRU_KEY = "GRU"
LSTM_KEY = "LSTM"

INPUT_LANG_KEY = "input_lang"
OUTPUT_LANG_KEY = "output_lang"
PAIRS_KEY = "pairs"
MAX_LEN_KEY = "max_len"

input_lang = "eng"
TARGET_LANG = "hin"

TRAIN_LABEL = "train"
TEST_LABEL = "test"
VALID_LABEL = "valid"

DEFAULT_PATH = "/kaggle/input/aksharantar-sampled/aksharantar_sampled"
TRAIN_DATASET_PATH = f"{DEFAULT_PATH}/{TARGET_LANG}/{TARGET_LANG}_{TRAIN_LABEL}.csv"
VALIDATION_DATASET_PATH = f"{DEFAULT_PATH}/{TARGET_LANG}/{TARGET_LANG}_{VALID_LABEL}.csv"
TEST_DATASET_PATH = f"{DEFAULT_PATH}/{TARGET_LANG}/{TARGET_LANG}_{TEST_LABEL}.csv"

NADAM_KEY = "Nadam"

# Sweep param labels
EMBEDDING_SIZE_KEY = "embedding_size"
EPOCHS_KEY = "epochs"
ENCODER_LAYER_KEY = "encoder_layers"
DECODER_LAYER_KEY = "decoder_layers"
HIDDEN_LAYER_KEY = "hidden_layer"
IS_BIDIRECTIONAL_KEY = "bidirectional"
DROPOUT_KEY = "dropout"
CELL_TYPE_KEY = "cell_type"
LEARNING_RATE_KEY = "learning_rate"
BATCH_SIZE_KEY = "batch_size"

# wandb constants
WANDB_PROJECT_NAME="dl-assignment-3"
WANDB_ENTITY_NAME="dl-1"

# wandb plot titles
TRAIN_ACCURACY_TITLE = "train_acc"
VALIDATION_ACCURACY_TITLE = "val_acc"
TEST_ACCURACY_TITLE = "test_acc"
TRAIN_LOSS_TITLE = "train_loss"
VALIDATION_LOSS_TITLE = "val_loss"
TEST_LOSS_TITLE = "test_loss"
